In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [23]:
model_path = '../models/pose_landmarker_heavy.task'
video_path = '../data/sample.mov'

In [24]:
import cv2
movie = cv2.VideoCapture(video_path)
movie_fps = movie.get(cv2.CAP_PROP_FPS)
movie_frames = movie.get(cv2.CAP_PROP_FRAME_COUNT)
initial_timestamp = 0

In [25]:
# STEP 1: Import the necessary modules.
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO,)

pose_landmarker = PoseLandmarker.create_from_options(options)

In [26]:
output = cv2.VideoWriter(
    '../data/mediapipe_dartslive.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    30, 
    (int(movie.get(cv2.CAP_PROP_FRAME_WIDTH)), int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)
while movie.isOpened():
    ret, frame = movie.read()
    if not ret:
        break
    frame_timestamp_ms = int(movie.get(cv2.CAP_PROP_POS_MSEC))
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    pose_landmarker_result = pose_landmarker.detect_for_video(mp_image, frame_timestamp_ms)
    annotated_image = draw_landmarks_on_image(frame, pose_landmarker_result)
    output.write(annotated_image)

movie.release()
output.release()

In [48]:
play_movie = cv2.VideoCapture('../data/20230514192027.mp4')
while play_movie.isOpened():
    ret, frame = play_movie.read()
    if not ret:
        break
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        play_movie.release()
        cv2.destroyAllWindows()
        break

: 

In [45]:
%pwd

'/Users/takayamahajime/development/pose_estimation/test_notebook'